In [1]:
!pip install faiss-gpu-cu12

In [2]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer
import faiss
import torch
from tqdm.auto import trange
import pickle
import re
import torch.functional as F

In [3]:
def parse_dialog(sentences):
    return '; '.join(sentences)

def prepare_text(s):
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request(x):
    return f"{x['dialog']} {x['question']} {x['choice_0']} {x['choice_1']}  {x['choice_2']}"

def make_df(path, is_test=False):
    data = pd.read_json(path,lines=True)
    df = []
    
    for i in range(len(data)):
        row = data.iloc[i]
        dialog = parse_dialog(row[0])
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': dialog,
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df)

In [4]:
train_data = make_df('data/train.statement.jsonl')
vall_data = make_df('data/dev.statement.jsonl')
test_data = make_df('data/test.statement.jsonl',is_test=True)

In [5]:
train_data['request'] = train_data.apply(parse_request,axis=1)
vall_data['request'] = vall_data.apply(parse_request,axis=1)
test_data['request'] = test_data.apply(parse_request,axis=1)

In [6]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def prepare_text(s):
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())
    
def cls_pooling(model_output):
    return model_output[0][:, 0]

@torch.no_grad()
def create_embeds(texts,model,tokenizer,batch_size=64,max_length=1024):
    features = []
    texts = [prepare_text(x) for x in texts]
    for i in trange(len(texts) // batch_size + 1):
        encs = tokenizer(
            texts[i*batch_size:(i+1)*batch_size],
            truncation=True,
            padding='longest',
            max_length=max_length,
            return_tensors='pt'
        )
        encs['input_ids'] = encs['input_ids'].cuda()
        encs['attention_mask'] = encs['attention_mask'].cuda()
        if 'token_type_ids' in encs.keys():
            encs['token_type_ids'] = encs['token_type_ids'].cuda()
        model_output = model(**encs)
        embeds = cls_pooling(model_output)#mean_pooling(model_output, encs['attention_mask'])
        embeds = torch.nn.functional.normalize(embeds, p=2, dim=1).detach().cpu().numpy()
        features.append(embeds)
    return np.concatenate(features,dtype=np.float32)

In [7]:
encoder = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5').cuda()
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:
graph_state = pickle.load(open('graph.pkl','rb'))

In [9]:
concepts = [prepare_text(s)  if type(s) == str else ' ' for s in graph_state['concepts_maper']]

In [44]:
with open('cleared_concepts.pkl','wb') as f:
    pickle.dump(concepts,f)

In [10]:
features = create_embeds(concepts,encoder,tokenizer,batch_size=512)

  0%|          | 0/1562 [00:00<?, ?it/s]

In [29]:
feats2 = pickle.load(open('faiss_embeds.pkl','rb'))

In [30]:
feats2

array([[-0.04070757, -0.01867592,  0.03030628, ..., -0.0709397 ,
         0.058196  ,  0.07076018],
       [-0.01453096, -0.03774516, -0.01253215, ..., -0.01442768,
         0.08658124,  0.03320973],
       [-0.02358458, -0.03156019,  0.06268256, ...,  0.04461925,
        -0.0070273 ,  0.06653491],
       ...,
       [-0.00125807, -0.07426448,  0.05829842, ..., -0.03413932,
        -0.03524177,  0.01722224],
       [ 0.02243753,  0.00452731, -0.00625741, ..., -0.01401754,
        -0.06785449,  0.03296914],
       [ 0.00481237, -0.02081296,  0.01319016, ..., -0.02960393,
        -0.0874484 , -0.01619786]], dtype=float32)

In [38]:
len(graph_state['concepts_maper_inv'])

799273

In [42]:
len(graph_state['concepts_maper'])

799272

In [12]:
index = faiss.IndexFlatIP(features.shape[1]) #use faiss.IndexFlatL2 for l2 distance
faiss.normalize_L2(features)
index.add(features)
res = faiss.StandardGpuResources()
index = faiss.index_cpu_to_gpu(res, 0, index)
index.ntotal

799272

In [13]:
train_features = create_embeds(train_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/58 [00:00<?, ?it/s]

In [14]:
test_features = create_embeds(test_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/17 [00:00<?, ?it/s]

In [15]:
val_features = create_embeds(vall_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/7 [00:00<?, ?it/s]

In [16]:
faiss.normalize_L2(train_features)
D_train,I_train = index.search(train_features,k=5)

In [17]:
faiss.normalize_L2(test_features)
D_test,I_test = index.search(test_features,k=5)

In [18]:
faiss.normalize_L2(val_features)
D_vall,I_vall = index.search(val_features,k=5)

In [19]:
train_data['candidates'] = [[concepts[j] for j in x] for x in I_train]
test_data['candidates'] = [[concepts[j] for j in x] for x in I_test]
vall_data['candidates'] = [[concepts[j] for j in x] for x in I_vall]

In [53]:
#!mkdir prep_data

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
train_data['candidates'].to_csv('./prep_data/train_dream_bge.csv',index=False)
test_data['candidates'].to_csv('./prep_data/test_dream_bge.csv',index=False)
vall_data['candidates'].to_csv('./prep_data/val_dream_bge.csv',index=False)

In [21]:
train_choices = train_data['choice_0'].tolist() + train_data['choice_1'].tolist() + train_data['choice_2'].tolist()
test_choices = test_data['choice_0'].tolist() + test_data['choice_1'].tolist() + test_data['choice_2'].tolist()
vall_choices = vall_data['choice_0'].tolist() + vall_data['choice_1'].tolist() + vall_data['choice_2'].tolist()

In [22]:
train_choice_embs = create_embeds(train_choices,encoder,tokenizer,batch_size=128,max_length=512)
test_choices_embs = create_embeds(test_choices,encoder,tokenizer,batch_size=128,max_length=512)
vall_choices_embs = create_embeds(vall_choices,encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/172 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [23]:
faiss.normalize_L2(train_choice_embs)
D_ch_train,I_ch_train = index.search(train_choice_embs,k=1)
faiss.normalize_L2(test_choices_embs)
D_ch_test,I_ch_test = index.search(test_choices_embs,k=1)
faiss.normalize_L2(vall_choices_embs)
D_ch_val,I_ch_val = index.search(vall_choices_embs,k=1)

In [24]:
train_data['choice_1_concept'] = [concepts[j] for j in I_ch_train[:,0][:len(I_ch_train)//3]]
train_data['choice_2_concept'] = [concepts[j] for j in I_ch_train[:,0][len(I_ch_train)//3:2*len(I_ch_train)//3]]
train_data['choice_3_concept'] = [concepts[j] for j in I_ch_train[:,0][2*len(I_ch_train)//3:]]

In [83]:
test_data['choice_1_concept'] = [concepts[j] for j in I_ch_test[:,0][:len(I_ch_test)//3]]
test_data['choice_2_concept'] = [concepts[j] for j in I_ch_test[:,0][len(I_ch_test)//3:2*len(I_ch_test)//3]]
test_data['choice_3_concept'] = [concepts[j] for j in I_ch_test[:,0][2*len(I_ch_test)//3:]]

In [84]:
vall_data['choice_1_concept'] = [concepts[j] for j in I_ch_val[:,0][:len(I_ch_val)//3]]
vall_data['choice_2_concept'] = [concepts[j] for j in I_ch_val[:,0][len(I_ch_val)//3:2*len(I_ch_val)//3]]
vall_data['choice_3_concept'] = [concepts[j] for j in I_ch_val[:,0][2*len(I_ch_val)//3:]]

In [88]:
train_data[['choice_1_concept','choice_2_concept','choice_3_concept']].to_csv('./prep_data/train_choices.csv',index=False)
test_data[['choice_1_concept','choice_2_concept','choice_3_concept']].to_csv('./prep_data/test_choices.csv',index=False)
vall_data[['choice_1_concept','choice_2_concept','choice_3_concept']].to_csv('./prep_data/vall_choices.csv',index=False)

In [90]:
vall_data

dialog_id                                             dialog  \
0            0  M: I believe that choosing this man will be a ...   
1            1  M: Oh, no, it's Monday again. I always feel ti...   
2            1  M: Oh, no, it's Monday again. I always feel ti...   
3            1  M: Oh, no, it's Monday again. I always feel ti...   
4            2  M: Would you like something to drink, Mum?; W:...   
..         ...                                                ...   
808        515  Justin: Uh, hi. Do you need some help with tho...   
809        515  Justin: Uh, hi. Do you need some help with tho...   
810        515  Justin: Uh, hi. Do you need some help with tho...   
811        515  Justin: Uh, hi. Do you need some help with tho...   
812        515  Justin: Uh, hi. Do you need some help with tho...   

                                              question  \
0                        What is the woman's attitude?   
1                              What day was yesterday?   
2    How many people were invited to the party at L...   
3              With whom did the woman watch the film?   
4              What has the woman ordered for herself?   
..                                                 ...   
808                          What is the woman's name?   
809  The woman spent most of her life in __________...   
810               What does the woman do for a living?   
811            How often does the woman go to the gym?   
812  Which statement is NOT true about the conversa...   

                                     choice_0  \
0                    She agrees with the man.   
1                                     Monday.   
2                                  Seventeen.   
3                                Her brother.   
4                                       Milk.   
..                                        ...   
808                     Her name is Ruth Ann.   
809                                California   
810             She is a computer programmar.   
811        She goes there three times a week.   
812  The man is from a city called Salt Lake.   

                                      choice_1  \
0              She doesn't agree with the man.   
1                                      Sunday.   
2                                     Sixteen.   
3                             Her best friend.   
4                                       Juice.   
..                                         ...   
808                                It's Ronda.   
809                                       Utah   
810                     She works at a school.   
811           She works out four times a week.   
812  The woman gives the man her phone number.   

                                 choice_2  labels  \
0            She doesn't know what to do.       1   
1                                Tuesday.       1   
2                                 Twenty.       1   
3                          Her boyfriend.       0   
4                                 Coffee.       0   
..                                    ...     ...   
808           The woman's name is Rachel.       2   
809                              New York       1   
810              She has a job at a bank.       1   
811      She exercises five times a week.       2   
812  The woman knows a lot about history.       1   

                                               request  \
0    M: I believe that choosing this man will be a ...   
1    M: Oh, no, it's Monday again. I always feel ti...   
2    M: Oh, no, it's Monday again. I always feel ti...   
3    M: Oh, no, it's Monday again. I always feel ti...   
4    M: Would you like something to drink, Mum?; W:...   
..                                                 ...   
808  Justin: Uh, hi. Do you need some help with tho...   
809  Justin: Uh, hi. Do you need some help with tho...   
810  Justin: Uh, hi. Do you need some help with tho...   
811  Justin: Uh, hi. Do you need some help with tho...   
812  Justin: U